## Data Preprocessing and Integration

In [23]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

Anomaly detection 

In [24]:
def validate_data(df, df_name):
    """Validate and clean data, keeping first of duplicate pairs"""
    print(f"\nValidating {df_name}...")
    
    # Define duplicate criteria
    if df_name == "Ghana data":
        dup_cols = ['station', 'name', 'date']
    elif df_name == "ICCO data":
        dup_cols = ['date', 'price_usd_per_tonne']
    elif df_name == "FX data":
        dup_cols = ['date', 'Buying', 'Selling', 'Mid Rate']
    else:
        dup_cols = ['date']
    
    # Find duplicates
    duplicates = df[df.duplicated(dup_cols, keep=False)]
    
    if not duplicates.empty:
        print("Duplicate pairs:", duplicates.sort_values(dup_cols))
        
        # Keep first of each pair
        df = df.drop_duplicates(dup_cols, keep='first')
        print(f"Now duplicated rows: {len(df[df.duplicated(dup_cols)])}")
        # print(f"Num of rows with 2023-12-15 and price 4272.15: {len(df[(df['date'] == '2023-12-15') & (df['price_usd_per_tonne' == 4272.15)])}")
        # print(f"Num of rows with 2024-01-09 and price 4171.24: {len(df[(df['date'] == '2024-01-09') & (df['price_usd_per_tonne'] == 4171.24)])}")
    else:
        print("No duplicates found.")

Load and clean Ghana weather data

In [25]:
fx = pd.read_csv('dirty/fx_rates.csv')

numeric_cols = ['Buying', 'Selling', 'Mid Rate']
fx[numeric_cols] = fx[numeric_cols].replace(',', '', regex=True).astype(float)

fx['date'] = pd.to_datetime(fx['Date']).dt.strftime('%Y-%m-%d')
fx['date'] = pd.to_datetime(fx['date'])

fx = fx[['date', 'Buying', 'Selling', 'Mid Rate', 'Currency Pair']]
display(fx.head())

,date,Buying,Selling,Mid Rate,Currency Pair
0,2025-03-28,15.5222,15.5378,15.53,USDGHS
1,2025-03-27,15.5222,15.5378,15.53,USDGHS
2,2025-03-26,15.5222,15.5378,15.53,USDGHS
3,2025-03-25,15.5122,15.5278,15.52,USDGHS
4,2025-03-24,15.5122,15.5278,15.52,USDGHS


In [26]:
ghana = pd.read_csv('dirty/Ghana_data.csv')
ghana = ghana.rename(columns={
    'STATION': 'station',
    'NAME': 'name',
    'DATE': 'date',
    'PRCP': 'precipitation',
    'TMAX': 'max_temp',
    'TMIN': 'min_temp',
    'TAVG': 'avg_temp',
})
ghana['date'] = pd.to_datetime(ghana['date'])

Load and clean ICCO price data

In [27]:
icco_prices = pd.read_csv(
    'dirty/Daily_Prices_ICCO.csv',
    thousands=',',
    decimal='.',
    parse_dates=['Date'],
    dayfirst=True
)
icco_prices = icco_prices.rename(columns={
    'Date': 'date',
    'ICCO daily price (US$/tonne)': 'price_usd_per_tonne',
})

Validate data quality

In [28]:
validate_data(ghana, "Ghana data")
validate_data(icco_prices, "ICCO data")
validate_data(fx, "FX data")


Validating Ghana data...
No duplicates found.

Validating ICCO data...
Duplicate pairs:           date  price_usd_per_tonne
310 2023-12-15              4272.15
311 2023-12-15              4272.15
295 2024-01-09              4171.24
296 2024-01-09              4171.24
Now duplicated rows: 0

Validating FX data...
No duplicates found.


Date coverage analysis

In [29]:
date_coverage = pd.DataFrame({
    'Data Source': ['Ghana', 'ICCO', 'FX'],
    'Start Date': [ghana['date'].min(), icco_prices['date'].min(), fx['date'].min()],
    'End Date': [ghana['date'].max(), icco_prices['date'].max(), fx['date'].max()],
    'Unique Dates': [ghana['date'].nunique(), icco_prices['date'].nunique(), fx['date'].nunique()]
})

min_date = date_coverage['Start Date'].max()
max_date = date_coverage['End Date'].min()

print("\nDate Coverage Analysis:")
print(f"\n{date_coverage.to_string()}")
print(f"\nOverall Date Range: {min_date} to {max_date}")


Date Coverage Analysis:

  Data Source Start Date   End Date  Unique Dates
0       Ghana 1990-01-01 2024-11-28         10944
1        ICCO 1994-10-03 2025-02-27          7808
2          FX 1996-01-02 2025-03-28          7279

Overall Date Range: 1996-01-02 00:00:00 to 2024-11-28 00:00:00


Combine multiple station weather data to one station

In [30]:
ghana.sort_values(by='date', inplace=True)
ghana.to_csv('dirty/sorted_ghana.csv', index=False)

combined_weather = ghana.groupby('date', as_index=False).agg({
    'precipitation': 'mean',
    'avg_temp': 'mean',
    'max_temp': 'mean',
    'min_temp': 'mean'
}).round(5)

validate_data(combined_weather, "Combined Ghana data")

combined_weather['year_month'] = combined_weather['date'].dt.to_period('M')  # e.g., "1990-01"
combined_weather.to_csv('dirty/monthly_weather.csv', index=False)


Validating Combined Ghana data...
No duplicates found.


### Convert all individual daily data to monthly data

In [31]:
monthly_weather = (
    combined_weather.groupby(pd.Grouper(key='date', freq='M'))  # Group by month
    .agg({
        'precipitation': ['mean', 'sum', 'count'],  # Total rain, max daily rain, # of days
        'max_temp': ['mean', 'max'],              # Avg and highest max temp
        'min_temp': ['mean', 'min'],             # Avg and lowest min temp
        'avg_temp': ['mean', 'std']               # Avg temp and variability
    })
    .pipe(lambda x: x.set_axis(
        ['_'.join(col).strip() for col in x.columns],  # Flatten column names
        axis=1
    ))
    .reset_index()
)
monthly_weather['date'] = monthly_weather['date'].dt.strftime('%Y-%m')  # Format date as string
monthly_weather = monthly_weather.rename(columns={'date': 'year_month'})
display(monthly_weather.head())

/var/folders/3m/2lzp5b051qd01070jvyf9zw80000gn/T/ipykernel_8284/3479515266.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  combined_weather.groupby(pd.Grouper(key='date', freq='M'))  # Group by month


,year_month,precipitation_mean,precipitation_sum,precipitation_count,max_temp_mean,max_temp_max,min_temp_mean,min_temp_min,avg_temp_mean,avg_temp_std
0,1990-01,0.097500,0.39000,4,90.200000,92.0,75.478261,71.0,81.483333,1.534002
1,1990-02,0.147143,1.03000,7,91.150000,94.0,75.264706,68.0,82.172619,2.607161
2,1990-03,0.086750,0.69400,8,94.431818,100.0,77.409091,71.0,84.723118,1.706929
3,1990-04,0.350000,5.60000,16,91.416667,94.0,75.017544,70.0,82.103448,2.241458
4,1990-05,0.234542,4.69084,20,89.196429,92.0,74.117361,68.0,81.018818,2.380934


In [32]:
monthly_prices = (
    icco_prices
    .groupby(pd.Grouper(key='date', freq='M'))
    .agg({
        'price_usd_per_tonne': ['mean', 'std']
    })
    .pipe(lambda x: x.set_axis(
        ['_'.join(col).strip() for col in x.columns],
        axis=1
    ))
    .reset_index()
)
monthly_prices['date'] = monthly_prices['date'].dt.strftime('%Y-%m')  # Format date as string
monthly_prices = monthly_prices.rename(columns={'date': 'year_month'})
display(monthly_prices.head())

/var/folders/3m/2lzp5b051qd01070jvyf9zw80000gn/T/ipykernel_8284/370489567.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .groupby(pd.Grouper(key='date', freq='M'))


,year_month,price_usd_per_tonne_mean,price_usd_per_tonne_std
0,1994-10,1447.960952,27.457564
1,1994-11,1436.980455,31.719475
2,1994-12,1399.390476,32.800616
3,1995-01,1468.818095,26.684452
4,1995-02,1510.437500,42.214454


In [33]:
monthly_fx = (
    fx.groupby(pd.Grouper(key='date', freq='M'))
    .agg({
        'Mid Rate': ['mean', 'std']
    })
    .pipe(lambda x: x.set_axis(
        ['_'.join(col).strip() for col in x.columns],
        axis=1
    ))
    .reset_index()
)

monthly_fx['year_month'] = monthly_fx['date'].dt.strftime('%Y-%m')
monthly_fx = monthly_fx.drop(columns=['date'])
monthly_fx = monthly_fx[['year_month'] + [col for col in monthly_fx.columns if col != 'year_month']]
display(monthly_fx.head())

/var/folders/3m/2lzp5b051qd01070jvyf9zw80000gn/T/ipykernel_8284/2618525594.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  fx.groupby(pd.Grouper(key='date', freq='M'))


,year_month,Mid Rate_mean,Mid Rate_std
0,1996-01,1469.078409,14.536908
1,1996-02,1518.639000,13.286225
2,1996-03,1561.531500,11.793660
3,1996-04,1595.029250,8.399923
4,1996-05,1618.618636,7.556588


check if there is any duplicated or missing year_month

In [34]:
def missing_month(df, name):
    # Convert string 'year_month' to datetime (first day of each month)
    df_dates = pd.to_datetime(df['year_month'] + '-01')
    
    # Generate all expected months in the date range
    full_date_range = pd.date_range(
        start=df_dates.min(),
        end=df_dates.max(),
        freq='MS'  # Month Start frequency
    ).to_period('M')
    
    # Convert existing dates to Period for comparison
    existing_months = pd.to_datetime(df['year_month'] + '-01').dt.to_period('M')
    
    # Case 1: Months completely absent from data
    missing_months = full_date_range[~full_date_range.isin(existing_months)]
    
    # Case 2: Months present but with either:
    # - All NaN values, OR
    # - Combination of NaN and 0 (no meaningful data)
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df_numeric = df[numeric_cols]
    
    # Identify rows where all numeric columns are either NaN or 0
    empty_months_mask = (
        df_numeric.isna() |  # True for NaN values
        (df_numeric == 0)    # True for zero values
    ).all(axis=1)  # True if ALL columns are zero/NaN for that row
    
    # Get the year_month values for these empty months
    empty_months = df.loc[empty_months_mask, 'year_month']
    empty_months = pd.to_datetime(empty_months + '-01').dt.to_period('M')
    
    # Combine results (convert to sets to avoid duplicates)
    all_missing = set(missing_months) | set(empty_months)
    all_missing = pd.PeriodIndex(sorted(all_missing), freq='M')
    
    if not all_missing.empty:
        print(f"Missing or empty months in {name}:")
        print(all_missing.strftime('%Y-%m').tolist(), "\n")
    else:
        print("No entire months are missing or empty.", "\n")

In [35]:
missing_month(monthly_weather, "monthly weather data")
missing_month(monthly_prices, "monthly prices data")
missing_month(monthly_fx, "monthly fx data")

Missing or empty months in monthly weather data:
['1994-12', '2001-10', '2001-12'] 

No entire months are missing or empty. 

Missing or empty months in monthly fx data:
['2006-06'] 



In [36]:
print(monthly_fx['year_month'].duplicated().any())
print(monthly_prices['year_month'].duplicated().any())
print(monthly_weather['year_month'].duplicated().any())

False
False
False


## Merge the datasets on date

In [37]:
display(monthly_weather.head())

,year_month,precipitation_mean,precipitation_sum,precipitation_count,max_temp_mean,max_temp_max,min_temp_mean,min_temp_min,avg_temp_mean,avg_temp_std
0,1990-01,0.097500,0.39000,4,90.200000,92.0,75.478261,71.0,81.483333,1.534002
1,1990-02,0.147143,1.03000,7,91.150000,94.0,75.264706,68.0,82.172619,2.607161
2,1990-03,0.086750,0.69400,8,94.431818,100.0,77.409091,71.0,84.723118,1.706929
3,1990-04,0.350000,5.60000,16,91.416667,94.0,75.017544,70.0,82.103448,2.241458
4,1990-05,0.234542,4.69084,20,89.196429,92.0,74.117361,68.0,81.018818,2.380934


In [38]:
# Convert date columns to datetime for proper merging
monthly_weather['date'] = pd.to_datetime(monthly_weather['year_month'] + '-01')
monthly_prices['date'] = pd.to_datetime(monthly_prices['year_month'] + '-01')
monthly_fx['date'] = pd.to_datetime(monthly_fx['year_month'] + '-01')

# Inner merge (keep only dates present in all three datasets)
merged_inner = (
    monthly_weather.merge(monthly_prices, on='date', how='inner')
    .merge(monthly_fx, on='date', how='inner')
    .sort_values('date')
    .drop(columns=['year_month_x', 'year_month_y'])
    .reset_index(drop=True)
)

# Drop unnecessary columns
merged_inner['year'] = merged_inner['date'].dt.year
merged_inner['month'] = merged_inner['date'].dt.month
merged_inner.drop(columns=['date'], inplace=True)

col_order = ['year_month', 'year', 'month'] + [col for col in merged_inner.columns if col not in ['year_month', 'year', 'month']]
merged_inner = merged_inner[col_order]

display(merged_inner.head())
missing_month(merged_inner, "merged data")

,year_month,year,month,precipitation_mean,precipitation_sum,precipitation_count,max_temp_mean,max_temp_max,min_temp_mean,min_temp_min,avg_temp_mean,avg_temp_std,price_usd_per_tonne_mean,price_usd_per_tonne_std,Mid Rate_mean,Mid Rate_std
0,1996-01,1996,1,NaN,0.00000,0,NaN,NaN,75.500000,75.00000,82.166667,1.940790,1349.541364,10.184784,1469.078409,14.536908
1,1996-02,1996,2,NaN,0.00000,0,NaN,NaN,NaN,NaN,84.285714,1.603567,1369.839524,17.896659,1518.639000,13.286225
2,1996-03,1996,3,NaN,0.00000,0,90.500000,91.0,NaN,NaN,84.000000,2.070197,1338.600952,26.267301,1561.531500,11.793660
3,1996-04,1996,4,NaN,0.00000,0,NaN,NaN,75.000000,75.00000,83.666667,3.511885,1456.279524,31.037661,1595.029250,8.399923
4,1996-05,1996,5,0.414524,5.80333,14,87.480556,91.6,74.803571,70.66667,80.691111,2.745478,1524.020455,17.324733,1618.618636,7.556588


No entire months are missing or empty. 



In [39]:
def handle_missing(df):
    """Impute missing values with appropriate methods"""
    print("\nHandling missing values...")
    
    # Check for any rows with NaN values in any column
    rows_with_nans = df[df.isnull().any(axis=1)]
    if not rows_with_nans.empty:
        print(f"\nFound {len(rows_with_nans)} rows with NaN values:")
        display(rows_with_nans.head())
    else:
        print("\nNo rows with NaN values found in the dataframe")
    
    # Show only columns with missing values
    df_numeric = df.select_dtypes(include=['float64', 'int64'])
    missing_before = df_numeric.isnull().sum()
    missing_before = missing_before[missing_before > 0]
    print(f"\nMissing values per column before handling:\n{missing_before}")

    # Precipitation: 0 for missing (assuming no rain)
    df['precipitation_mean'] = df['precipitation_mean'].fillna(0)
    
    # Temperatures: linear interpolation for missing
    for col in ['max_temp_mean', 'max_temp_max',
                'min_temp_mean', 'min_temp_min',
                'avg_temp_mean', 'avg_temp_std',
                ]:
        df[col] = df[col].interpolate(method='linear', limit_direction='both')
        
    # Show missing values after handling
    missing_after = df.isnull().sum()
    missing_after = missing_after[missing_after > 0]
    print(f"\nMissing values per column after handling:\n{missing_after}")
    
    # Verify no NaN values remain
    if df.isnull().any().any():
        print("\nWarning: Some NaN values remain after handling!")
    else:
        print("\nAll NaN values successfully handled")
    
    return df

In [40]:
def check_anomalies(df):
    """Check for data anomalies"""
    print("\nRunning Data Quality Checks:")
    
    # Temperature logical consistency
    try:
        assert (df['max_temp_mean'] >= df['avg_temp_mean']).all(), "Max temp mean < Avg temp mean"
        assert (df['avg_temp_mean'] >= df['min_temp_mean']).all(), "Avg temp mean  < Min temp mean"
        print("✓ Temperature consistency checks passed")
    except AssertionError as e:
        print(f"Temperature anomaly: {e}")
    
    # Precipitation non-negative
    try:
        assert (df['precipitation_mean'] >= 0).all(), "Negative precipitation"
        print("✓ Precipitation non-negativity check passed")
    except AssertionError as e:
        print(f"Precipitation anomaly: {e}")
    
    # Price positive
    try:
        assert (df['price_usd_per_tonne_mean'] > 0).all(), "Non-positive prices"
        print("✓ Price positivity check passed")
    except AssertionError as e:
        print(f"Price anomaly: {e}")

In [41]:
handle_missing(merged_inner)
check_anomalies(merged_inner)


Handling missing values...

Found 43 rows with NaN values:


,year_month,year,month,precipitation_mean,precipitation_sum,precipitation_count,max_temp_mean,max_temp_max,min_temp_mean,min_temp_min,avg_temp_mean,avg_temp_std,price_usd_per_tonne_mean,price_usd_per_tonne_std,Mid Rate_mean,Mid Rate_std
0,1996-01,1996,1,NaN,0.0,0,NaN,NaN,75.500000,75.0,82.166667,1.940790,1349.541364,10.184784,1469.078409,14.536908
1,1996-02,1996,2,NaN,0.0,0,NaN,NaN,NaN,NaN,84.285714,1.603567,1369.839524,17.896659,1518.639000,13.286225
2,1996-03,1996,3,NaN,0.0,0,90.500000,91.0,NaN,NaN,84.000000,2.070197,1338.600952,26.267301,1561.531500,11.793660
3,1996-04,1996,4,NaN,0.0,0,NaN,NaN,75.000000,75.0,83.666667,3.511885,1456.279524,31.037661,1595.029250,8.399923
36,1999-01,1999,1,NaN,0.0,0,89.789474,93.0,74.896825,72.0,81.979167,1.545541,1455.232000,23.304263,2317.007632,151.766155



Missing values per column before handling:
precipitation_mean    41
max_temp_mean         15
max_temp_max          15
min_temp_mean         16
min_temp_min          16
avg_temp_mean          2
avg_temp_std           6
dtype: int64

Missing values per column after handling:
Series([], dtype: int64)

All NaN values successfully handled

Running Data Quality Checks:
✓ Temperature consistency checks passed
✓ Precipitation non-negativity check passed
✓ Price positivity check passed


## Normalization

In [42]:
def normalize_weather_data(df):
    """Properly normalize weather variables by annual baselines"""
    print("\nNormalizing weather data using Z-score standardization...")
    
    # Identify weather columns to normalize
    weather_cols = [
        'precipitation_mean', 'precipitation_sum',
        'max_temp_mean', 'max_temp_max',
        'min_temp_mean', 'min_temp_min',
        'avg_temp_mean', 'avg_temp_std'
    ]
    
    # Create normalized versions
    scaler = StandardScaler()
    weather_data = df[weather_cols]
    
    # Fit and transform the data
    normalized_data = scaler.fit_transform(weather_data)
    df[weather_cols] = normalized_data
    col_order = ["year_month", "year", "month"] + weather_cols + [col for col in df.columns if col not in weather_cols + ["year_month", "year", "month"]]
    df = df[col_order]
    
    return df

In [43]:
merged_inner.to_csv('dirty/all_before_norm.csv', index=False)
merged_inner = normalize_weather_data(merged_inner)
merged_inner.to_csv('dirty/all_after_norm.csv', index=False)
display(merged_inner.head())


Normalizing weather data using Z-score standardization...


,year_month,year,month,precipitation_mean,precipitation_sum,max_temp_mean,max_temp_max,min_temp_mean,min_temp_min,avg_temp_mean,avg_temp_std,precipitation_count,price_usd_per_tonne_mean,price_usd_per_tonne_std,Mid Rate_mean,Mid Rate_std
0,1996-01,1996,1,-0.861464,-1.011531,0.462460,-0.238712,0.947776,1.934933,0.438983,0.376641,0,1349.541364,10.184784,1469.078409,14.536908
1,1996-02,1996,2,-0.861464,-1.011531,0.462460,-0.238712,0.838467,1.934933,1.344259,-0.256773,0,1369.839524,17.896659,1518.639000,13.286225
2,1996-03,1996,3,-0.861464,-1.011531,0.462460,-0.238712,0.729157,1.934933,1.222199,0.619708,0,1338.600952,26.267301,1561.531500,11.793660
3,1996-04,1996,4,-0.861464,-1.011531,0.016635,-0.155776,0.619848,1.934933,1.079796,3.327665,0,1456.279524,31.037661,1595.029250,8.399923
4,1996-05,1996,5,0.786976,0.665290,-0.429190,-0.072839,0.491019,0.027792,-0.191387,1.888106,14,1524.020455,17.324733,1618.618636,7.556588


## Train Test Split

In [44]:
simplified_cols = [
    'year_month', 'year', 'month',
    'precipitation_sum',
    'max_temp_mean',
    'min_temp_mean',
    'avg_temp_mean',
    'price_usd_per_tonne_mean',
    'Mid Rate_mean',
]

# ====== TIME-BASED SPLIT ======
def time_series_split(df, test_ratio=0.2):
    """Split DataFrame chronologically into train and test sets"""
    df = df.sort_values('year_month')  # Ensure chronological order
    split_idx = int(len(df) * (1 - test_ratio))
    return df.iloc[:split_idx], df.iloc[split_idx:]

# Split only the full dataset
train_full, test_full = time_series_split(merged_inner)

# Create simplified versions FROM THE SPLIT DATA
train_simple = train_full[simplified_cols]
test_simple = test_full[simplified_cols]

# ====== VERIFICATION ======
def print_date_ranges(df, name):
    df['year_month'] = pd.to_datetime(df['year_month'] + '-01')
    min_date = f"{df['year_month'].min().year}-{df['year_month'].min().month:02d}"
    max_date = f"{df['year_month'].max().year}-{df['year_month'].max().month:02d}"
    print(f"{name}: {min_date} to {max_date} ({len(df)} months)")

print("\nDate Range Verification:")
print_date_ranges(train_full, "Train Full")
print_date_ranges(test_full, "Test Full") 

# ====== SAVING ======
datasets = {
    'train_full.csv': train_full,
    'test_full.csv': test_full,
    'train_simple.csv': train_simple,
    'test_simple.csv': test_simple
}

for filename, data in datasets.items():
    data.to_csv(filename, index=False)
    print(f"Saved {filename}")

print("\nAll data processed and split successfully.")


Date Range Verification:
Train Full: 1996-01 to 2019-01 (277 months)
Test Full: 2019-02 to 2024-11 (70 months)
Saved train_full.csv
Saved test_full.csv
Saved train_simple.csv
Saved test_simple.csv

All data processed and split successfully.
